In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import ResNet50
# from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50
size = 64
batchsize = 512
lw = 2
channel = 1
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [3]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [4]:
K.clear_session()
# base_model = MobileNet(input_shape=(size, size, 3), alpha=1., weights='imagenet', include_top=False)
# x = base_model.output
# # x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# x = Dense(64)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# predictions = Dense(NCATS, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

model = ResNet50(input_shape=(size, size, 1), weights=None, classes=NCATS)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 1) (34000, 340)
Validation array memory 0.52 GB


In [7]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [8]:
model_prefix = 'resnet50_raw_head'
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}{}.model".format(model_prefix, size),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}{}'.format(model_prefix, size)),
]

In [9]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/50
647/646 [==============================] - 283s 437ms/step - loss: 3.4457 - categorical_crossentropy: 3.4457 - categorical_accuracy: 0.2826 - top_3_accuracy: 0.4506 - val_loss: 12.9870 - val_categorical_crossentropy: 12.9870 - val_categorical_accuracy: 0.0117 - val_top_3_accuracy: 0.4203

Epoch 00001: val_categorical_accuracy improved from -inf to 0.01171, saving model to ./models/resnet50_raw_head64.model
Epoch 2/50
647/646 [==============================] - 274s 424ms/step - loss: 2.0902 - categorical_crossentropy: 2.0902 - categorical_accuracy: 0.5080 - top_3_accuracy: 0.7150 - val_loss: 6.1859 - val_categorical_crossentropy: 6.1859 - val_categorical_accuracy: 0.0539 - val_top_3_accuracy: 0.1320

Epoch 00002: val_categorical_accuracy improved from 0.01171 to 0.05391, saving model to ./models/resnet50_raw_head64.model
Epoch 3/50
647/646 [==============================] - 274s 424ms/step - loss: 1.6886 - categorical_crossentropy: 1.6886 - categorical_accuracy: 0.5925 - top_

Epoch 22/50
647/646 [==============================] - 275s 425ms/step - loss: 1.1703 - categorical_crossentropy: 1.1703 - categorical_accuracy: 0.7117 - top_3_accuracy: 0.8725 - val_loss: 1.3712 - val_categorical_crossentropy: 1.3712 - val_categorical_accuracy: 0.6662 - val_top_3_accuracy: 0.8439

Epoch 00022: val_categorical_accuracy did not improve from 0.71303
Epoch 23/50
647/646 [==============================] - 275s 425ms/step - loss: 1.2334 - categorical_crossentropy: 1.2334 - categorical_accuracy: 0.6969 - top_3_accuracy: 0.8626 - val_loss: 15.5330 - val_categorical_crossentropy: 15.5330 - val_categorical_accuracy: 0.0067 - val_top_3_accuracy: 0.3704

Epoch 00023: val_categorical_accuracy did not improve from 0.71303
Epoch 24/50
647/646 [==============================] - 275s 424ms/step - loss: 1.1267 - categorical_crossentropy: 1.1267 - categorical_accuracy: 0.7180 - top_3_accuracy: 0.8747 - val_loss: 1.1652 - val_categorical_crossentropy: 1.1652 - val_categorical_accuracy: 0


Epoch 00042: val_categorical_accuracy improved from 0.75362 to 0.76471, saving model to ./models/resnet50_raw_head64.model
Epoch 43/50
647/646 [==============================] - 275s 425ms/step - loss: 0.9325 - categorical_crossentropy: 0.9325 - categorical_accuracy: 0.7625 - top_3_accuracy: 0.9011 - val_loss: 0.9323 - val_categorical_crossentropy: 0.9323 - val_categorical_accuracy: 0.7632 - val_top_3_accuracy: 0.9022

Epoch 00043: val_categorical_accuracy did not improve from 0.76471
Epoch 44/50
647/646 [==============================] - 275s 425ms/step - loss: 0.9280 - categorical_crossentropy: 0.9280 - categorical_accuracy: 0.7637 - top_3_accuracy: 0.9020 - val_loss: 0.9428 - val_categorical_crossentropy: 0.9428 - val_categorical_accuracy: 0.7616 - val_top_3_accuracy: 0.8995

Epoch 00044: val_categorical_accuracy did not improve from 0.76471
Epoch 45/50
647/646 [==============================] - 275s 425ms/step - loss: 0.9226 - categorical_crossentropy: 0.9226 - categorical_accurac

In [10]:
model.load_weights('./models/{}{}.model'.format(model_prefix, size))

In [11]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

(112199, 3) (112199, 64, 64, 1)
Test array memory 1.71 GB


In [12]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

112199/112199 [==============================] - 37s 328us/step


(112199, 3)

In [13]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

(112199, 3)

In [14]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

,key_id,word
0,9000003627287624,radio stereo snorkel
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle fence
3,9000038588854897,mountain tent The_Eiffel_Tower
4,9000052667981386,fireplace campfire crown


In [15]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}{}.csv'.format(model_prefix, size), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)

save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/resnet50_raw_head64.csv.7z -m "submit"
